In [16]:
import os
import requests


In [17]:
os.listdir()

['wiki', '.ipynb_checkpoints', 'Basics.ipynb', 'scrape_random.py']

In [18]:
print('Total of files in the wiki folder:', len(os.listdir('wiki')))
print()
os.listdir('wiki')
      
      

Total of files in the wiki folder: 999



['Furubira_District,_Hokkaido.html',
 'Valentin_Yanin.html',
 'Kings_XI_Punjab_in_2014.html',
 'William_Harvey_Lillard.html',
 'Radial_Road_3.html',
 'George_Weldrick.html',
 'Zgornji_Otok.html',
 'Blue_Heelers_(season_8).html',
 'Taggen_Nunatak.html',
 '1951_National_League_tie-breaker_series.html',
 'List_of_number-one_singles_of_1993_(Finland).html',
 'Vrila.html',
 'William_Henry_Porter.html',
 'Clive_Brown_(footballer).html',
 '2010_Karshi_Challenger_%E2%80%93_Singles.html',
 'Blick_nach_Rechts.html',
 'Central_District_(Rezvanshahr_County).html',
 'Gal%C3%A1pagos,_Guadalajara.html',
 'Campus_of_Texas_A%26M_University.html',
 'Alexios_Aspietes.html',
 'Mei_Lanfang.html',
 'Thalkirchen-Obersendling-Forstenried-F%C3%BCrstenried-Solln.html',
 'Coalville_Town_railway_station.html',
 'Gennady_Lesun.html',
 'Bartrum_Glacier.html',
 'Victor_S._Mamatey.html',
 'Gottfried_Keller.html',
 'Table_Point_Formation.html',
 'Nobuhiko_Ushiba.html',
 'Master_of_Space_and_Time.html',
 'Early_medieva

In [19]:
with open('wiki/Valentin_Yanin.html') as f:
    print(f.read())


<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Valentin Yanin - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Valentin_Yanin","wgTitle":"Valentin Yanin","wgCurRevisionId":736718079,"wgRevisionId":736718079,"wgArticleId":7502081,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles containing Russian-language text","Wikipedia articles with VIAF identifiers","Wikipedia articles with LCCN identifiers","Wikipedia articles with BNF identifiers","Living people","1929 births","Russian historians","Soviet historians","20th-century historians","Russian archaeologists","Soviet archaeologists","Moscow

<b>Read all of the files into a list called content 
using different methods<b>

In [20]:
import time
import concurrent.futures
import os

content = []
articles = []

start = time.time()
for f in os.listdir('wiki'):
    with open('wiki/' + f) as f:
        content.append(f.read())
    articles.append(f)
    
elapsed = time.time() - start
print('Total time 1 thread', elapsed)

# ------------------------------------------------------------

def read_file_content(filename):
    
    content = []
    
    with open('wiki/' + filename) as f:
        content.append(f.read())
   
    return content, filename

#read_file_content('Taggen_Nunatak.html')
start = time.time()

pool = concurrent.futures.ThreadPoolExecutor(max_workers=3)
files_content = pool.map(read_file_content, os.listdir("wiki"))

files_content = list(files_content)

articles = []
content = []

for f in files_content:
    content.append(f[0])
    articles.append(f[1])
    
elapsed = time.time() - start
print('Total time when processing with 3 threads', elapsed)

# ------------------------------------------------------------
start = time.time()

pool = concurrent.futures.ProcessPoolExecutor(max_workers=3)
files_content = pool.map(read_file_content, os.listdir("wiki"))

files_content = list(files_content)

articles = []
content = []

for f in files_content:
    content.append(f[0])
    articles.append(f[1])
    
elapsed = time.time() - start
print('Total time when processing with 3 processors', elapsed)




Total time 1 thread 0.11574244499206543
Total time when processing with 3 threads 0.5025150775909424
Total time when processing with 3 processors 0.9935586452484131


<b>Parse each file and extract all the content inside of the div with id content<b>

In [21]:
from bs4 import BeautifulSoup
import os
import time

def read_file_content(filename):
    
    with open('wiki/' + filename) as f:
        parser = BeautifulSoup(f.read(), 'html.parser')

        # Pass in the ID attribute to only get the element 
        # with that specific ID.
        div_content = parser.find_all("div", id="content")[0]
        
    return(str(div_content))

# print(read_file_content('Taggen_Nunatak.html'))

start = time.time()

pool = concurrent.futures.ProcessPoolExecutor(max_workers=3)
files_content = pool.map(read_file_content, os.listdir("wiki"))
files_content = list(files_content)

parsed =[f for f in files_content]

print(len(parsed))

elapsed = time.time() - start
print('Total time when processing with 3 processors', elapsed)





999
Total time when processing with 3 processors 41.276649475097656


In [22]:
from bs4 import BeautifulSoup
import os
import time
from collections import Counter

def read_file_content(filename):
    
    with open('wiki/' + filename) as f:
        parser = BeautifulSoup(f.read(), 'html.parser')

        # Pass in the ID attribute to only get the element 
        # with that specific ID.
        div_content = parser.find_all("div", id="content")[0].find_all(True)
        
        # Get the tag names only
        content = [t.name for t in div_content]
        
        # Count number of tags in the file
        tag_count = Counter(content)
    return(dict(tag_count))

 
# print(read_file_content('Taggen_Nunatak.html'))

start = time.time()

pool = concurrent.futures.ProcessPoolExecutor(max_workers=3)
files_content = pool.map(read_file_content, os.listdir("wiki"))
files_content = list(files_content)

# Combine all file's tags into one dictionary
tag_count = {}
for f in files_content:
    for k, v in f.items():
        if k not in tag_count.keys():
            tag_count[k] = 0
        
        tag_count[k] += v
        
    
print(Counter(tag_count).most_common(200))

elapsed = time.time() - start
print()
print('Total time when processing with 3 processors', elapsed)





[('a', 161065), ('li', 85779), ('span', 67350), ('td', 57673), ('div', 27582), ('tr', 27300), ('i', 18246), ('th', 14472), ('b', 14455), ('sup', 11157), ('ul', 10972), ('p', 7998), ('img', 6701), ('br', 4986), ('h2', 4045), ('table', 4010), ('abbr', 3665), ('cite', 3563), ('small', 3272), ('dd', 1376), ('h1', 999), ('noscript', 999), ('ol', 858), ('h3', 777), ('strong', 599), ('dl', 457), ('dt', 334), ('caption', 200), ('sub', 151), ('h4', 117), ('code', 108), ('wbr', 85), ('q', 76), ('big', 75), ('center', 64), ('blockquote', 58), ('u', 51), ('hr', 51), ('font', 40), ('area', 39), ('rp', 32), ('rb', 16), ('ruby', 16), ('rt', 16), ('s', 10), ('bdi', 4), ('h5', 4), ('samp', 2), ('map', 2), ('mrow', 2), ('annotation', 2), ('source', 2), ('del', 2), ('mstyle', 2), ('math', 2), ('semantics', 2), ('audio', 2), ('mo', 2), ('pre', 1), ('h6', 1)]

Total time when processing with 3 processors 34.011722564697266


<b> The "a" tag is the most used (161,065) among the 999 pages. This shows pages on 
Wikipedia references each other frequently <b>

In [43]:
from bs4 import BeautifulSoup
import os
import time
from collections import Counter
import re

def read_file_content(filename):
    
    with open('wiki/' + filename) as f:
        parser = BeautifulSoup(f.read(), 'html.parser')

        # Pass in the ID attribute to only get the element 
        # with that specific ID.
        div_content = parser.find_all("div", id="content")[0].find_all(True)
        
        # Get the text inside the div only
        # lower case all words
        # Remove non-alphanumerical characters
        # only return words that have more than 5 characters
        content = []
        for t in div_content:
            t = re.sub("\W+", " ", t.text.lower())
            words = t.split(" ")
        
            for w in words:
                if len(w)>=5:
                    content.append(w)
        
        # Count number of words in the file
        words_count = Counter(content)
            
    return dict(words_count)
      
# print(read_file_content('Taggen_Nunatak.html'))

start = time.time()

pool = concurrent.futures.ProcessPoolExecutor(max_workers=3)
files_content = pool.map(read_file_content, os.listdir("wiki"))
files_content = list(files_content)

word_count = {}
for f in files_content:
    for k,v in f.items():
        if k not in word_count.keys():
            word_count[k] = 0
        word_count[k] = v
        
top_100 = Counter(word_count).most_common(100)
print(top_100)

print('Total time to process with 3 processes = ', time.time()-start)
            



print('done')


[('bellator', 2868), ('lille', 1639), ('rangers', 994), ('mickey', 653), ('laboratoire', 604), ('dragnet', 599), ('antibiotics', 593), ('rockefeller', 574), ('области', 562), ('amazon', 517), ('coups', 506), ('seyyed', 494), ('rushes', 480), ('kampala', 426), ('ellington', 424), ('marvel', 412), ('sheehan', 403), ('braille', 387), ('mazzucato', 377), ('tubular', 376), ('manhattan', 369), ('bandholtz', 358), ('административно', 351), ('petrolero', 350), ('pushkar', 336), ('закон', 336), ('westchester', 334), ('forbath', 333), ('hawke', 333), ('subgenius', 325), ('morgana', 324), ('опубликован', 322), ('mattel', 320), ('hackensaw', 311), ('tasman', 310), ('вступил', 308), ('cybernetics', 307), ('amborella', 302), ('jones', 300), ('cyclone', 298), ('sartana', 298), ('institut', 294), ('aikido', 294), ('waitakere', 293), ('внесении', 287), ('amending', 287), ('django', 285), ('sudanese', 285), ('bounty', 278), ('acorah', 277), ('cocker', 269), ('christos', 266), ('изменений', 266), ('templ